In [6]:
import pandas as pd
from faker import Faker
import random
import numpy as np

# --- CONFIGURAÇÃO ---
NUM_DOCTORS = 20
NUM_PATIENTS = 50
NUM_DEALS = 100
DUPLICATE_PATIENTS_COUNT = 3
PATIENTS_WITH_MISSING_LASTNAME = 5
DEALS_WITHOUT_DOCTOR = 10

# Inicializa o Faker
fake = Faker('pt_BR')

def generate_doctors(num_doctors):
    """Gera dados fictícios para médicos (HubSpot Companies)."""
    specialties = ['Neurologia', 'Cardiologia', 'Clínico Geral', 'Psiquiatria', 'Ortopedia', 'Neuro'] # 'Neuro' é um erro proposital
    data = []
    for _ in range(num_doctors):
        name = fake.name_male()
        # Erro Proposital 1: Formatação inconsistente na especialidade
        industry = random.choice(specialties) if random.random() > 0.1 else 'Cardiologia'
        
        data.append({
            'name': f"Dr(a). {name}",
            'domain': f"{name.split(' ')[-1].lower()}.med.br",
            'industry': industry
        })
    
    # Erro Proposital 2: Duplicata de Médico
    if num_doctors > 0:
        data.append(data[0])

    return pd.DataFrame(data)

def generate_patients(num_patients):
    """Gera dados fictícios para pacientes (HubSpot Contacts)."""
    data = []
    for i in range(num_patients):
        first_name = fake.first_name()
        last_name = fake.last_name()
        
        # Erro Proposital 3: Dados Faltando (sobrenome)
        if i < PATIENTS_WITH_MISSING_LASTNAME:
            last_name = None
            
        data.append({
            'firstname': first_name,
            'lastname': last_name,
            'email': f"{first_name.lower()}.{last_name.lower() if last_name else ''}@{fake.free_email_domain()}"
        })
        
    # Erro Proposital 4: Duplicatas de Pacientes
    for i in range(DUPLICATE_PATIENTS_COUNT):
        if data:
            data.append(data[i])
            
    return pd.DataFrame(data)

def generate_deals(num_deals, df_doctors, df_patients):
    """Gera dados fictícios para negócios (HubSpot Deals), associando médicos e pacientes."""
    deal_stages = ['consulta_agendada', 'receita_emitida', 'medicamento_comprado', 'compra_recorrente', 'cancelado']
    data = []
    
    # Garante que temos pacientes e médicos para associar
    if df_patients.empty or df_doctors.empty:
        return pd.DataFrame()

    for i in range(num_deals):
        patient = df_patients.sample(1).iloc[0]
        doctor = df_doctors.sample(1).iloc[0]
        
        # Erro Proposital 5: Associações Ausentes (negócio sem médico)
        company_domain = doctor['domain'] if i >= DEALS_WITHOUT_DOCTOR else None
        
        data.append({
            'dealname': f"Atendimento - {patient['firstname']}",
            'amount': round(random.uniform(100, 800), 2),
            'dealstage': random.choice(deal_stages),
            'contact_email': patient['email'],
            'company_domain': company_domain
        })
        
    return pd.DataFrame(data)

if __name__ == "__main__":
    print("Gerando dados fictícios...")
    
    df_doctors = generate_doctors(NUM_DOCTORS)
    df_patients = generate_patients(NUM_PATIENTS)
    df_deals = generate_deals(NUM_DEALS, df_doctors, df_patients)
    
    # Salva em arquivos CSV prontos para importação
    df_doctors.to_csv('import_companies.csv', index=False)
    df_patients.to_csv('import_contacts.csv', index=False)
    df_deals.to_csv('import_deals.csv', index=False)
    
    print("\n--- Resumo dos Dados Gerados ---")
    print(f"Médicos (Companies): {len(df_doctors)} registros salvos em 'import_companies.csv'")
    print(f"Pacientes (Contacts): {len(df_patients)} registros salvos em 'import_contacts.csv'")
    print(f"Negócios (Deals): {len(df_deals)} registros salvos em 'import_deals.csv'")
    
    print("\n--- Problemas de Qualidade Injetados ---")
    print(f"-> {len(df_doctors[df_doctors.duplicated()])} Médico(s) duplicado(s).")
    print(f"-> Formatos de especialidade inconsistentes (ex: 'Neurologia' vs 'Neuro').")
    print(f"-> {DUPLICATE_PATIENTS_COUNT} Paciente(s) duplicado(s).")
    print(f"-> {PATIENTS_WITH_MISSING_LASTNAME} Paciente(s) com sobrenome faltando.")
    print(f"-> {DEALS_WITHOUT_DOCTOR} Negócio(s) sem médico associado.")
    
    print("\nPróximo passo: Importe os arquivos CSV para o HubSpot na seguinte ordem: 1. companies, 2. contacts, 3. deals.")

Gerando dados fictícios...

--- Resumo dos Dados Gerados ---
Médicos (Companies): 21 registros salvos em 'import_companies.csv'
Pacientes (Contacts): 53 registros salvos em 'import_contacts.csv'
Negócios (Deals): 100 registros salvos em 'import_deals.csv'

--- Problemas de Qualidade Injetados ---
-> 1 Médico(s) duplicado(s).
-> Formatos de especialidade inconsistentes (ex: 'Neurologia' vs 'Neuro').
-> 3 Paciente(s) duplicado(s).
-> 5 Paciente(s) com sobrenome faltando.
-> 10 Negócio(s) sem médico associado.

Próximo passo: Importe os arquivos CSV para o HubSpot na seguinte ordem: 1. companies, 2. contacts, 3. deals.
